In [1]:
import pandas as pd
import os
import sys
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

with open('../config/repository_path.txt', 'r') as reader:
    repository_path=reader.read().strip()
sys.path.append(repository_path)

from lib import models
from lib import dataset
from lib import tdmstudio

In [15]:
data={}
for file in os.listdir(os.path.join(repository_path, 'data','predictions')):
    df = pd.read_csv(os.path.join(repository_path, 'data','predictions',file))
    for id_, score in zip(df['id'], df['score']):
        if not id_ in data:
            data[id_]=score
        else:
            assert data[id_]-score<1e-6, f'{data[id_]},{score}'
    
ids=list(data)
all_scores = pd.DataFrame({'id':ids, 'score':[data[id_] for id_ in data]})
all_scores = all_scores.sort_values(by='score', ascending=False)
all_scores.head()

,id,score
1105503,2610146759,0.750947
50858,1143646857,0.668956
1109735,1241441332,0.663298
1107436,1125784644,0.653163
1107399,1151111154,0.650276


In [30]:
ranges = [2,
          4,
          7,
          14,
          27,
          53,
          102,
          197,
          381,
          737,
          1426,
          2759,
          5340,
          10335,
          20000,
         ]

count_per_range = [2,
                   2,
                   2,
                   2,
                   3,
                   3,
                   3,
                   4,
                   5,
                   7,
                   8,
                   10,
                   12,
                   17,
                   20,
                  ]
begin = 0
selected_dfs=[]
for range_no, (end, size) in enumerate(zip(ranges, count_per_range)):
    auxdf = all_scores.iloc[begin:end]

    ix = np.random.choice(range(len(auxdf)), size=size, replace=False)

    selected = auxdf.iloc[ix,:].copy()
    selected['range']=range_no
    selected_dfs.append(selected)

    begin=end
to_label = pd.concat(selected_dfs)
to_label.head()

,id,score,range
1105503,2610146759,0.750947,0
50858,1143646857,0.668956,0
1107436,1125784644,0.653163,1
1109735,1241441332,0.663298,1
1109580,1513508940,0.636664,2


In [37]:
files =  list(map(tdmstudio.get_filename, [str(id_) for id_ in to_label['id']]))
to_label['title'] =list(map(tdmstudio.get_title, files))
to_label.to_csv(os.path.join(repository_path, 'data', '100_articles_evaluation_20k_suggestions_1st_round_scal_refugee.csv'),
                index=False
               )

In [36]:
to_label

,id,score,range,title
1105503,2610146759,0.750947,0,The refugee issue is dangerous politics.
50858,1143646857,0.668956,0,Refugees and immigration.
1107436,1125784644,0.653163,1,"Crack down on refugees, Commons report urges."
1109735,1241441332,0.663298,1,Broaden definition of refugee in immigration b...
1109580,1513508940,0.636664,2,A plan to make Canada less desireable.
...,...,...,...,...
3570390,1135470046,0.243836,14,PEOPLE FOR PROFIT.
400068,1288488037,0.228679,14,Canada Still Cruelly Racial.
4232988,1270025137,0.247374,14,Wisely done.
2255707,1287701646,0.253839,14,New Measure Would Cut Delays In Deportations.


In [23]:
all_score

,id,score
747560,1237170136,0.633891
1109748,1238331892,0.632148
1112884,1143733555,0.627192
1106202,2122118681,0.626680
1120418,1269996330,0.622261
1121214,1444926563,0.621348
1105421,2748166774,0.618836
